In [1]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
# instance_id 样本编号
# item_id 广告商品编号
# item_category_list 广告商品的的类目列表 分割; item_property_list_0 item_property_list_1 item_property_list_2
# item_property_list 广告商品的属性列表 分割 1 2 3
# item_brand_id 广告商品的品牌编号
# item_city_id 广告商品的城市编号
# item_price_level 广告商品的价格等级
# item_sales_level 广告商品的销量等级
# item_collected_level 广告商品被收藏次数的等级
# item_pv_level 广告商品被展示次数的等级
# user_id 用户的编号
# 'user_gender_id', 用户的预测性别编号
# 'user_age_level', 用户的预测年龄等级
# 'user_occupation_id', 用户的预测职业编号
# 'user_star_level' 用户的星级编号
# context_id 上下文信息的编号
#  context_timestamp 广告商品的展示时间
# context_page_id 广告商品的展示页面编号
# predict_category_property
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']


#     print('predict_category_property_ing_0')
#     for i in range(3):
#         data['predict_category_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[0] if len(x.split(";")) > i else " "
#         )
#     for i in range(3):
#         temp = data[['user_occupation_id','property_%d'%(i),'instance_id']].groupby(['user_occupation_id','property_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_occupation_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['property_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_occupation_id','property_%d'%(i)],how='left')
#     for i in range(1,3):
#         temp = data[['user_occupation_id','category_%d'%(i),'instance_id']].groupby(['user_occupation_id','category_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_occupation_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['category_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_occupation_id','category_%d'%(i)],how='left')
#     for i in range(3):
#         temp = data[['user_gender_id','property_%d'%(i),'instance_id']].groupby(['user_gender_id','property_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_gender_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['property_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_gender_id','property_%d'%(i)],how='left')
#     for i in range(1,3):
#         temp = data[['user_gender_id','category_%d'%(i),'instance_id']].groupby(['user_gender_id','category_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_gender_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['category_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_gender_id','category_%d'%(i)],how='left')
#     temp = data[['user_age_level','item_price_level','instance_id']].groupby(['user_age_level','item_price_level']).aggregate('count')
#     temp1 = temp.reset_index()
#     temp2 = temp.groupby('user_age_level').sum().reset_index()
#     temp3 = pd.merge(temp1,temp2,on=['user_age_level'],how='left')
#     temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#     temp3['age_pv'] = temp4
#     temp3.drop('instance_id_x', axis=1, inplace=True)
#     temp3.drop('instance_id_y', axis=1, inplace=True)
#     data = pd.merge(data,temp3,on=['user_age_level','user_age_level'],how='left')
#     del(temp3)
#     del(temp2)
#     del(temp1)
#     del(temp4)
#     del(temp)
#     print('predict_category_property_ing_1')
#     for i in range(3):
#         data['predict_property_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[1] if len(x.split(";")) > i else " "
#         )
    
#         for j in range(3):
#             data['predict_property_%d_%d' % (i,j)] = data['predict_property_%d'%(i)].apply(
#                 lambda x: x.split(",")[j] if len(x.split(",")) > j else -1
#             )

    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']

    return data



In [2]:
print('train')
train = pd.read_csv('train.csv',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False) 
train = pd.merge(train,dataa,on=['instance_id'],how='left')
train = pre_process(train)



train
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [3]:
all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (478087, 39)
2018-09-24 23:59:47
(357066, 39)
(121021, 39)


In [4]:

print('test')
test_a = pd.read_csv('test.csv',index_col=False)
data2a = pd.read_csv('test_all.csv',index_col=False) 
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']
# 0.83108
# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test
(18371, 34)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [5]:

import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count




In [6]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)
# print(train.columns)
train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


2018-09-18 00:00:01
2018-09-16 00:00:01
NaT
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43
(18371, 37)


In [7]:

print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
# print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
# feat_set = list(test_a.columns)

del train['user_id']
del test_a['user_id']
del val['user_id']
# 8288

print(train.columns)

# 自己使用onehot编码
train['x_train'] = 1
test_a['x_test'] = 1
val['x_val'] = 1

all_data = pd.concat([train,test_a,val])
all_data = pd.get_dummies(all_data)
print(all_data.shape)

X_train = all_data.loc[all_data['x_train'].notnull()]
X_test = all_data.loc[all_data['x_test'].notnull()]
X_val = all_data.loc[all_data['x_val'].notnull()]

del X_train['x_train']
del X_train['x_test']
del X_train['x_val']
del X_test['x_train']
del X_test['x_test']
del X_test['x_val']
del X_val['x_train']
del X_val['x_test']
del X_val['x_val']


baseline ing ... ...
Index(['item_id', 'item_brand_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_page_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'item_category_list_num',
       'item_property_list_num', 'hour_num', 'category_1_pvnum',
       'property_0_pvnum', 'property_1_pvnum', 'property_2_pvnum',
       'item_id_pvnum', 'category_0', 'category_1', 'category_2', 'property_0',
       'property_1', 'property_2', 'halfhour', 'user_count', 'item_count'],
      dtype='object')
(496458, 590)


In [8]:
# LogisticRegression?

In [9]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [10]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [15]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=50,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 1
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.683901
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674841
[3]	valid_0's binary_logloss: 0.665956
[4]	valid_0's binary_logloss: 0.657239
[5]	valid_0's binary_logloss: 0.648693
[6]	valid_0's binary_logloss: 0.640309
[7]	valid_0's binary_logloss: 0.632089
[8]	valid_0's binary_logloss: 0.624022
[9]	valid_0's binary_logloss: 0.6161
[10]	valid_0's binary_logloss: 0.608327
[11]	valid_0's binary_logloss: 0.600695
[12]	valid_0's binary_logloss: 0.593201
[13]	valid_0's binary_logloss: 0.585846
[14]	valid_0's binary_logloss: 0.578625
[15]	valid_0's binary_logloss: 0.57153
[16]	valid_0's binary_logloss: 0.564563
[17]	valid_0's binary_logloss: 0.557711
[18]	valid_0's binary_logloss: 0.55098
[19]	valid_0's binary_logloss: 0.544375
[20]	valid_0's binary_logloss: 0.537884
[21]	valid_0's binary_logloss: 0.531503
[22]	valid_0's binary_logloss: 0.525216
[23]	valid_0's binary_logloss: 0.519045
[24]	valid_0's binary_logloss:

[204]	valid_0's binary_logloss: 0.12518
[205]	valid_0's binary_logloss: 0.124636
[206]	valid_0's binary_logloss: 0.124101
[207]	valid_0's binary_logloss: 0.123568
[208]	valid_0's binary_logloss: 0.123047
[209]	valid_0's binary_logloss: 0.122531
[210]	valid_0's binary_logloss: 0.122019
[211]	valid_0's binary_logloss: 0.121514
[212]	valid_0's binary_logloss: 0.121017
[213]	valid_0's binary_logloss: 0.120524
[214]	valid_0's binary_logloss: 0.120037
[215]	valid_0's binary_logloss: 0.119558
[216]	valid_0's binary_logloss: 0.119087
[217]	valid_0's binary_logloss: 0.11862
[218]	valid_0's binary_logloss: 0.118156
[219]	valid_0's binary_logloss: 0.117699
[220]	valid_0's binary_logloss: 0.117246
[221]	valid_0's binary_logloss: 0.116801
[222]	valid_0's binary_logloss: 0.116361
[223]	valid_0's binary_logloss: 0.115926
[224]	valid_0's binary_logloss: 0.1155
[225]	valid_0's binary_logloss: 0.115075
[226]	valid_0's binary_logloss: 0.114657
[227]	valid_0's binary_logloss: 0.114243
[228]	valid_0's bina

[407]	valid_0's binary_logloss: 0.0855145
[408]	valid_0's binary_logloss: 0.0854762
[409]	valid_0's binary_logloss: 0.0854374
[410]	valid_0's binary_logloss: 0.0853969
[411]	valid_0's binary_logloss: 0.0853571
[412]	valid_0's binary_logloss: 0.0853221
[413]	valid_0's binary_logloss: 0.0852852
[414]	valid_0's binary_logloss: 0.0852485
[415]	valid_0's binary_logloss: 0.0852137
[416]	valid_0's binary_logloss: 0.0851775
[417]	valid_0's binary_logloss: 0.0851435
[418]	valid_0's binary_logloss: 0.0851087
[419]	valid_0's binary_logloss: 0.0850729
[420]	valid_0's binary_logloss: 0.0850405
[421]	valid_0's binary_logloss: 0.0850093
[422]	valid_0's binary_logloss: 0.0849774
[423]	valid_0's binary_logloss: 0.0849452
[424]	valid_0's binary_logloss: 0.084913
[425]	valid_0's binary_logloss: 0.0848813
[426]	valid_0's binary_logloss: 0.0848506
[427]	valid_0's binary_logloss: 0.0848212
[428]	valid_0's binary_logloss: 0.0847929
[429]	valid_0's binary_logloss: 0.0847653
[430]	valid_0's binary_logloss: 0.0

[606]	valid_0's binary_logloss: 0.0826237
[607]	valid_0's binary_logloss: 0.082619
[608]	valid_0's binary_logloss: 0.0826134
[609]	valid_0's binary_logloss: 0.0826087
[610]	valid_0's binary_logloss: 0.0826051
[611]	valid_0's binary_logloss: 0.0826
[612]	valid_0's binary_logloss: 0.0825952
[613]	valid_0's binary_logloss: 0.0825909
[614]	valid_0's binary_logloss: 0.0825887
[615]	valid_0's binary_logloss: 0.0825838
[616]	valid_0's binary_logloss: 0.0825795
[617]	valid_0's binary_logloss: 0.0825755
[618]	valid_0's binary_logloss: 0.0825701
[619]	valid_0's binary_logloss: 0.0825672
[620]	valid_0's binary_logloss: 0.0825649
[621]	valid_0's binary_logloss: 0.0825633
[622]	valid_0's binary_logloss: 0.082559
[623]	valid_0's binary_logloss: 0.0825556
[624]	valid_0's binary_logloss: 0.0825519
[625]	valid_0's binary_logloss: 0.0825495
[626]	valid_0's binary_logloss: 0.0825484
[627]	valid_0's binary_logloss: 0.0825451
[628]	valid_0's binary_logloss: 0.082542
[629]	valid_0's binary_logloss: 0.082538

[803]	valid_0's binary_logloss: 0.0822748
[804]	valid_0's binary_logloss: 0.0822736
[805]	valid_0's binary_logloss: 0.0822718
[806]	valid_0's binary_logloss: 0.082271
[807]	valid_0's binary_logloss: 0.0822715
[808]	valid_0's binary_logloss: 0.082273
[809]	valid_0's binary_logloss: 0.0822706
[810]	valid_0's binary_logloss: 0.0822706
[811]	valid_0's binary_logloss: 0.0822695
[812]	valid_0's binary_logloss: 0.0822683
[813]	valid_0's binary_logloss: 0.0822667
[814]	valid_0's binary_logloss: 0.0822658
[815]	valid_0's binary_logloss: 0.0822651
[816]	valid_0's binary_logloss: 0.082265
[817]	valid_0's binary_logloss: 0.0822645
[818]	valid_0's binary_logloss: 0.0822643
[819]	valid_0's binary_logloss: 0.0822623
[820]	valid_0's binary_logloss: 0.0822622
[821]	valid_0's binary_logloss: 0.082262
[822]	valid_0's binary_logloss: 0.0822617
[823]	valid_0's binary_logloss: 0.0822618
[824]	valid_0's binary_logloss: 0.0822622
[825]	valid_0's binary_logloss: 0.082261
[826]	valid_0's binary_logloss: 0.08226

[999]	valid_0's binary_logloss: 0.0821837
[1000]	valid_0's binary_logloss: 0.0821852
[1001]	valid_0's binary_logloss: 0.0821864
[1002]	valid_0's binary_logloss: 0.0821877
[1003]	valid_0's binary_logloss: 0.0821868
[1004]	valid_0's binary_logloss: 0.082186
[1005]	valid_0's binary_logloss: 0.0821844
[1006]	valid_0's binary_logloss: 0.0821849
[1007]	valid_0's binary_logloss: 0.0821829
[1008]	valid_0's binary_logloss: 0.0821801
[1009]	valid_0's binary_logloss: 0.0821806
[1010]	valid_0's binary_logloss: 0.0821791
[1011]	valid_0's binary_logloss: 0.0821788
[1012]	valid_0's binary_logloss: 0.0821803
[1013]	valid_0's binary_logloss: 0.0821817
[1014]	valid_0's binary_logloss: 0.0821829
[1015]	valid_0's binary_logloss: 0.0821812
[1016]	valid_0's binary_logloss: 0.0821791
[1017]	valid_0's binary_logloss: 0.0821779
[1018]	valid_0's binary_logloss: 0.0821779
[1019]	valid_0's binary_logloss: 0.082179
[1020]	valid_0's binary_logloss: 0.0821772
[1021]	valid_0's binary_logloss: 0.0821787
[1022]	valid_0

[1196]	valid_0's binary_logloss: 0.0821531
[1197]	valid_0's binary_logloss: 0.0821524
[1198]	valid_0's binary_logloss: 0.0821525
[1199]	valid_0's binary_logloss: 0.0821517
[1200]	valid_0's binary_logloss: 0.0821529
[1201]	valid_0's binary_logloss: 0.0821521
[1202]	valid_0's binary_logloss: 0.0821526
[1203]	valid_0's binary_logloss: 0.0821535
[1204]	valid_0's binary_logloss: 0.0821538
[1205]	valid_0's binary_logloss: 0.0821529
[1206]	valid_0's binary_logloss: 0.0821533
[1207]	valid_0's binary_logloss: 0.082155
[1208]	valid_0's binary_logloss: 0.0821549
[1209]	valid_0's binary_logloss: 0.0821566
[1210]	valid_0's binary_logloss: 0.0821564
[1211]	valid_0's binary_logloss: 0.082157
[1212]	valid_0's binary_logloss: 0.0821558
[1213]	valid_0's binary_logloss: 0.0821553
[1214]	valid_0's binary_logloss: 0.082156
[1215]	valid_0's binary_logloss: 0.0821564
[1216]	valid_0's binary_logloss: 0.0821562
[1217]	valid_0's binary_logloss: 0.0821554
[1218]	valid_0's binary_logloss: 0.0821538
[1219]	valid_0

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=50, objective='binary', random_state=None,
       reg_alpha=1, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [16]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0816995451384

Start predicting...
0.0795379037051
0.082148899194


In [13]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [14]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180406.txt',sep=" ",index=False)